In [1]:
import torch 
import torchvision
import torchvision.transforms as transforms
from torchvision import models
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import confusion_matrix
import itertools
import numpy as np 
import matplotlib.pyplot as plt


In [2]:
transform=transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485,0.456,0.406],std=[0.229,0.224,0.225])
    
])

In [3]:
#load dataset
trainset=torchvision.datasets.CIFAR10(root='./data',train=True,download=True,transform=transform)
train_loader=torch.utils.data.DataLoader(trainset,batch_size=32,shuffle=True)



#Test Dataset
testset=torchvision.datasets.CIFAR10(root='./data',train=False,download=True,transform=transform)
test_loader=torch.utils.data.DataLoader(testset,batch_size=32,shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [4]:
vgg16=models.vgg16(pretrained=False)

#modify the classifier
num_features=vgg16.classifier[6].in_features
features=list(vgg16.classifier.children())[:-1]
features.extend([nn.Linear(num_features,len(trainset.classes))])
vgg16.classifier=nn.Sequential(*features)   #replace classification layer 


device=torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
vgg16=vgg16.to(device)



C:\Users\jaher5\Desktop\Neural Networks\myvenv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\jaher5\Desktop\Neural Networks\myvenv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [5]:
criterion=nn.CrossEntropyLoss()
optimizer=optim.SGD(vgg16.parameters(),lr=0.001,momentum=0.9)

In [ ]:
num_epochs = 10
losses = []
accuracies = []

for epoch in range(num_epochs):
    running_loss = 0.0
    correct = 0
    total = 0

    for i, data in enumerate(train_loader, 0):  # Iterate over the training data
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to the right device (GPU or CPU)

        optimizer.zero_grad()  # Zero out the gradients from the previous step

        outputs = vgg16(inputs)  # Forward pass: get model predictions
        loss = criterion(outputs, labels)  # Calculate loss
        loss.backward()  # Backpropagate the loss to compute gradients
        optimizer.step()  # Update model parameters based on gradients

        running_loss += loss.item()  # Add the current batch's loss to the running loss

        _, predicted = torch.max(outputs.data, 1)  # Get the predicted class label (index of max value)
        total += labels.size(0)  # Increment the total number of samples
        correct += (predicted == labels).sum().item()  # Increment the count of correct predictions

    # Calculate average loss and accuracy for the epoch
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = 100 * correct / total

    # Append the values for later use
    losses.append(epoch_loss)
    accuracies.append(epoch_accuracy)

    # Print loss and accuracy for this epoch
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_accuracy:.2f}%')


In [ ]:
torch.save(vgg16.state_dict(),"./vgg16_without_transfer_learning.pth")